In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# DeepSeek on VKS documents

## Using PostgreSQL to retrieval VKS documents


Define global variables

In [15]:
import os

from langchain_community.llms import VLLMOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from modules import utils

from IPython.display import display, Markdown, Latex

In [16]:
ENV_FILE_PATH = "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700
COLLECTION_NAME = "my_docs"

Load environment variables

In [17]:
envs = utils.load_env_to_dict(ENV_FILE_PATH)

## Prepare PostgreSQL vectorstore

In [18]:
from langchain_postgres.vectorstores import PGVector
from langchain_huggingface.embeddings import HuggingFaceEmbeddings


In [19]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)

In [20]:
vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=envs["POSTGRESQL_URI_2"],
    use_jsonb=True,
)

## Retrieval

In [21]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [22]:
res = vector_store.similarity_search("VKS là gì?")
utils.print_documents(res)

Amount of documents: 4
Document 0:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luô

## Integrated with DeepSeek R1

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [24]:
# Connect to vLLM API
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base=envs['VLLM_HOST_URL_2'] + "/v1/",
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    streaming=True,
)

prompt = ChatPromptTemplate.from_template("""
Bạn là một trợ lí ảo nhằm trả lời các CÂU HỎI cho khách hàng của VNGCloud.
Câu trả lời của bạn phải có liên quan đến NGỮ CẢNH được cung cấp. Nếu không liên quan hãy yêu cầu cung cấp thêm thông tin.
Bạn PHẢI trả lời bằng TIẾNG VIỆT

CÂU HỎI: {query}

NGỮ CẢNH:
{context}

TRẢ LỜI:
""")

chain = prompt | llm

## Prepare the context

In [25]:
query = "VKS là gì?"
context = utils.from_documents_to_context(vector_store.similarity_search(query))

display(Markdown(context[:1000]))

- Document 1:
VKS là gì?

VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các ứng dụng container trên môi trường phân tán.

Những điểm nổi bật của VKS

Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất (minor version từ 1.27, 1.28, 1.29) để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.

Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.

Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng, giúp bạn luôn cập nhật những cải 

In [26]:
query = "VKS là gì?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được rồi, mình đang cố gắng trả lời câu hỏi "VKS là gì?" bằng cách xem qua các tài liệu được cung cấp. Đầu tiên, mình chú ý đến Document 1 và 2, chúng đều giải thích VKS là VNGCloud Kubernetes Service, một dịch vụ quản lý trên VNGCloud giúp triển khai và quản lý ứng dụng container dựa trên Kubernetes.

Mình thấy rằng VKS tự động quản lý Control Plane, hỗ trợ các phiên bản Kubernetes mới nhất, tích hợp các công nghệ như Calico CNI và CSI. Ngoài ra, VKS còn có khả năng tự động scale và heal, giúp người dùng tiết kiệm thời gian quản lý. Mình cũng chú ý đến các cập nhật gần đây trong Document 3, như hỗ trợ giám sát tài nguyên, nâng cấp qua Terraform, và các cải tiến về bảo mật, hiệu suất mạng.

Document 4 ayrıntı hơn về việc sử dụng Terraform với VKS, hướng dẫn các bước khởi tạo cluster và node group. Điều này cho thấy VKS không chỉ là một dịch vụ McC 通리 mà còn tích hợp tốt với các công cụ DevOps như Terraform, giúp tự động hóa quá trình triển khai.

Khi tổng hợp thông tin, mìn

In [27]:
query = "Làm sao để expose service trong VKS?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Okay, so the user is asking how to expose a service in VKS. I remember that in Kubernetes, exposing a service typically involves creating a Service object and then an Ingress or using a LoadBalancer type. But since VKS is a managed Kubernetes service by VNGCloud, I should check if there are any specific instructions or differences in their setup.

Looking at the provided documents, especially Document 2 and Document 4, they talk about setting up Terraform configurations for creating clusters and node groups. They mention enabling load balancer plugins, which suggests that LoadBalancer is supported.

In a typical Kubernetes setup, you can expose a service by changing its type to LoadBalancer, which would provision a load balancer from the cloud provider. Alternatively, you can create an Ingress resource to route traffic via HTTP/HTTPS.

I should outline the steps the user needs to take. First, they need to create a Kubernetes Deployment or ensure their application is running

In [28]:
query = "Hoàng Sa và Trường Sa là của ai?"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được yêu cầu trả lời về chủ quyền của Hoàng Sa và Trường Sa, tôi xem xét các tài liệu cung cấp. Tài liệu 1 chỉ dẫn tham khảo thêm, Document 2 giới hạn và hạn chế CSI, Document 3 giải thích về Label và Taint, còn Document 4 nói về CSI. Không có thông tin nào liên quan đến chủ đề địa lý chính trị này. Do đó, tôi không có đủ thông tin để trả lời câu hỏi và cần yêu cầu khách hàng cung cấp thêm chi tiết.



GENERATED RESPONSE


Câu hỏi của bạn không liên quan đến thông tin предоставлена в.ActionListener. esos documentos говорят о Kubernetes, Lable, Taint и CSI.

Пожалуйста, предоставьте больше информации или переформулируйте вопрос.

In [29]:
query = "Tạo cho tôi PVC 20Gb trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được yêu cầu tạo PVC 20Gb trong VKS, tôi xem xét các tài liệu hướng dẫn cách tạo Persistent Volume Claim (PVC) thông qua file YAML. Tài liệu Document 1 và Document 2 đều提到 việc tạo PVC bằng cách sử dụng YAML và kể cả thay đổi kích thước lưu trữ. Document 1 cũng có thông tin về việc thay đổi kích thước PVC tối thiểu xuống 1GB, chứng tỏ việc tạo PVC với kích thước cụ thể là được hỗ trợ.

 Document 2 chi tiết cách tạo PVC size 20Gi bằng cách thêm tham số `requests: storage: 20Gi` vào YAML. Hơn nữa, mẫu YAML trong Document 2 và Document 3 đều prima usando refute size 20Gi. Điều này cho thấy cách thức đúng để tạo PVC 20Gb là sử dụng YAML với tham số phù hợp.

 Tôi cũng chú ý到 việc thêm annotation `isPOC: "true"` trong YAML nếu muốn sử dụng ví POC. Nếu không rõ thì cần hỏi thêm, nhưng mặc định có thể tạo PVC với kích thước yêu cầu.



GENERATED RESPONSE


Dưới đây là hướng dẫn tạo PVC 20GB trong VKS thông qua YAML:

1. **Tạo file YAML** với nội dung như sau:

```yaml
apiVersion: 

In [30]:
query = "Làm sao để cấu hình volume type khi dùng pvc trong vks"

print("THINKING...")
for chunk in chain.stream({"query": query,
                           "context": utils.from_documents_to_context(vector_store.similarity_search(query))}):
    if chunk.content.strip() == "</think>":
        print("\n\n\nGENERATED RESPONSE")
        continue

    print(chunk.content, end="", flush=True)

THINKING...
Được nghiên cứu từ các documentation về tổng hợp của người dùng, tôi hiểu rằng để cấu hình volume type khi dùng PVC trong VKS, cần xác định storageClassName phù hợp.

Tôi xem qua các document và thấy được nhắc đến storageClassName như là một phần quan trọng trong cấu hình PVC. Trong Document 1, người dùng định nghĩa storageClassName là my-expansion-storage-class. Điều này cho thấy rằng khi tạo PVC, cần phải chỉ định storageClassName tương ứng với loại volume mong muốn.

Tôi cũng chú ý rằng trong Document 4, có đề cập đến việc thay đổi storage class mặc định từ IOPS 200 sang IOPS 3000. Điều này ám chỉ rằng người dùng có thể tạo các StorageClass khác nhau với các thông số như IOPS và khả năng lưu trữ không giới hạn.

Từ đó, tôi suy ra rằng để cấu hình volume type, người dùng cần:

1. Tạo một StorageClass với các tham số phù hợp, bao gồm type của volume như IOPS và kích thước lưu trữ.

2. Khi tạo PVC, chỉ định storageClassName này để đảm bảo rằng PVC sử dụng đúng loại volume.
